In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from datetime import datetime, timedelta

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set up plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Setup complete!")

Setup complete!


In [2]:
def load_netcdf_safe(file_path, decode_times_first=True):
    """
    Safely load NetCDF file with automatic fallback for time decoding issues
    """
    try:
        if decode_times_first:
            print("Attempting to load with time decoding...")
            ds = xr.open_dataset(file_path)
            print("✓ Successfully loaded with time decoding")
        else:
            raise ValueError("Skip first attempt")
            
    except (ValueError, OSError) as e:
        if "time units" in str(e) or "REFERENCE_DATE_TIME" in str(e):
            print("⚠ Time decoding failed, loading without time decoding...")
            ds = xr.open_dataset(file_path, decode_times=False)
            print("✓ Successfully loaded without time decoding")
        else:
            print(f"Error loading file: {e}")
            return None
    
    return ds

# Example usage
DATA_DIR = '../NetCDFDatafile/'

# List available files
files = [f for f in os.listdir(DATA_DIR) if f.endswith('.nc')]
print(f"Found {len(files)} NetCDF files:")
for i, file in enumerate(files[:5]):  # Show first 5 files
    print(f"  {i}: {file}")

# Select and load a file
if files:
    file_index = min(2, len(files)-1)  # Use index 2 or last available
    sample_file = files[file_index]
    file_path = os.path.join(DATA_DIR, sample_file)
    
    print(f"\nLoading: {sample_file}")
    ds = load_netcdf_safe(file_path)

Found 3 NetCDF files:
  0: ISAS15_SSS_20020115_fld_PSAL.nc
  1: ISAS15_SSS_20031215_fld_PSAL.nc
  2: ISAS20_ARGO_20110615_dat_DOXY.nc

Loading: ISAS20_ARGO_20110615_dat_DOXY.nc
Attempting to load with time decoding...
⚠ Time decoding failed, loading without time decoding...
✓ Successfully loaded without time decoding


In [8]:
def explore_dataset(ds):
    """Comprehensive dataset exploration"""
    
    print("="*60)
    print("DATASET OVERVIEW")
    print("="*60)
    print(ds)
    
    print("\n" + "="*60)
    print("BASIC INFORMATION")
    print("="*60)
    print(f"File size: {ds.nbytes / 1024**2:.2f} MB")
    print(f"Dimensions: {dict(ds.dims)}")
    print(f"Variables: {len(ds.data_vars)}")
    print(f"Coordinates: {len(ds.coords)}")
    
    print("\n" + "="*60)
    print("VARIABLES SUMMARY")
    print("="*60)
    for var_name, var in ds.data_vars.items():
        print(f"{var_name:15} | Shape: {str(var.shape):20} | Type: {var.dtype}")
        if hasattr(var, 'long_name'):
            print(f"{'':15} | Description: {var.long_name}")
        print()
    
    print("="*60)
    print("COORDINATES")
    print("="*60)
    for coord_name, coord in ds.coords.items():
        print(f"{coord_name:15} | Shape: {str(coord.shape):20} | Type: {coord.dtype}")
        if coord_name in ['latitude', 'longitude', 'lat', 'lon']:
            print(f"{'':15} | Range: [{coord.min().values:.4f}, {coord.max().values:.4f}]")
        print()

# Run exploration
if 'ds' in locals() and ds is not None:
    explore_dataset(ds)

DATASET OVERVIEW
<xarray.Dataset> Size: 181MB
Dimensions:              (N_PROF: 33635, N_LEVELS: 187)
Coordinates:
    JULD_datetime        (N_PROF) datetime64[ns] 269kB 2013-02-06T15:22:30 .....
Dimensions without coordinates: N_PROF, N_LEVELS
Data variables: (12/21)
    REFERENCE_DATE_TIME  |S16 16B ...
    DATA_TYPE            |S16 16B ...
    PLATFORM_NUMBER      (N_PROF) |S8 269kB ...
    WMO_INST_TYPE        (N_PROF) |S4 135kB ...
    PI_NAME              (N_PROF) |S64 2MB ...
    DATA_CENTRE          (N_PROF) |S2 67kB ...
    ...                   ...
    DOXY                 (N_PROF, N_LEVELS) float32 25MB ...
    DOXY_CLMN            (N_PROF, N_LEVELS) float32 25MB ...
    DOXY_CLSD            (N_PROF, N_LEVELS) float32 25MB ...
    DOXY_ERME            (N_PROF, N_LEVELS) float32 25MB ...
    DOXY_ERUR            (N_PROF, N_LEVELS) float32 25MB ...
    DOXY_RESI            (N_PROF, N_LEVELS) float32 25MB ...
Attributes:
    Conventions:       CF-1.4
    title:             Year

In [4]:
def handle_juld_time(ds, reference_date='1950-01-01'):
    """
    Convert JULD (Julian Day) variables to proper datetime
    """
    time_vars = ['JULD', 'juld', 'TIME', 'time']
    converted_vars = []
    
    for var_name in time_vars:
        if var_name in ds.variables:
            print(f"Found time variable: {var_name}")
            
            # Check current units
            units = ds[var_name].attrs.get('units', 'No units specified')
            print(f"  Current units: {units}")
            
            # If units are problematic, convert manually
            if 'REFERENCE_DATE_TIME' in units or 'since' not in units:
                print(f"  Converting {var_name} using reference date: {reference_date}")
                
                # Convert to datetime
                try:
                    # Method 1: Using pandas
                    time_converted = pd.to_datetime(ds[var_name].values, 
                                                  unit='D', 
                                                  origin=reference_date)
                    
                    # Create new coordinate
                    ds = ds.assign_coords({f'{var_name}_datetime': 
                                         ([dim for dim in ds[var_name].dims], time_converted)})
                    
                    # Update attributes
                    ds[f'{var_name}_datetime'].attrs['long_name'] = f'{var_name} as datetime'
                    ds[f'{var_name}_datetime'].attrs['standard_name'] = 'time'
                    ds[f'{var_name}_datetime'].attrs['original_units'] = units
                    ds[f'{var_name}_datetime'].attrs['reference_date'] = reference_date
                    
                    converted_vars.append(f'{var_name}_datetime')
                    print(f"  ✓ Created {var_name}_datetime")
                    
                except Exception as e:
                    print(f"  ✗ Failed to convert {var_name}: {e}")
            
            else:
                print(f"  Units look OK for {var_name}")
    
    return ds, converted_vars

# Apply JULD conversion
if 'ds' in locals() and ds is not None:
    print("\n" + "="*60)
    print("TIME VARIABLE PROCESSING")
    print("="*60)
    ds, time_vars = handle_juld_time(ds)


TIME VARIABLE PROCESSING
Found time variable: JULD
  Current units: days since REFERENCE_DATE_TIME
  Converting JULD using reference date: 1950-01-01
  ✓ Created JULD_datetime


In [6]:
def plot_variable_overview(ds, max_vars=6):
    """Create overview plots of main variables"""
    
    # Get numeric variables only
    numeric_vars = []
    for var_name, var in ds.data_vars.items():
        if np.issubdtype(var.dtype, np.number) and var.size > 1:
            numeric_vars.append(var_name)
    
    # Limit to max_vars for display
    vars_to_plot = numeric_vars[:max_vars]
    
    if not vars_to_plot:
        print("No suitable variables found for plotting")
        return
    
    # Create subplots
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, var_name in enumerate(vars_to_plot):
        ax = axes[i]
        
        try:
            var_data = ds[var_name]
            
            # Different plot types based on dimensions
            if var_data.ndim == 1:
                # Line plot for 1D data
                var_data.plot(ax=ax)
                ax.set_title(f'{var_name} (1D)')
                
            elif var_data.ndim == 2:
                # Heatmap for 2D data
                if var_data.size < 10000:  # Only if not too large
                    var_data.plot(ax=ax)
                    ax.set_title(f'{var_name} (2D)')
                else:
                    # Sample the data if too large
                    sample = var_data.isel({dim: slice(0, min(100, var_data.sizes[dim])) 
                                          for dim in var_data.dims})
                    sample.plot(ax=ax)
                    ax.set_title(f'{var_name} (2D - sampled)')
                    
            else:
                # For higher dimensions, plot a slice
                slice_dims = {dim: 0 for dim in var_data.dims[:-2]}
                var_slice = var_data.isel(slice_dims)
                var_slice.plot(ax=ax)
                ax.set_title(f'{var_name} (slice)')
                
        except Exception as e:
            ax.text(0.5, 0.5, f'Error plotting\n{var_name}:\n{str(e)[:50]}...', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{var_name} (error)')
    
    # Remove empty subplots
    for i in range(len(vars_to_plot), len(axes)):
        axes[i].remove()
    
    plt.tight_layout()
    plt.show()

def plot_geographic_data(ds):
    """Plot geographic data if lat/lon coordinates exist"""
    
    # Look for latitude/longitude coordinates
    lat_names = ['latitude', 'lat', 'LATITUDE', 'LAT']
    lon_names = ['longitude', 'lon', 'LONGITUDE', 'LON']
    
    lat_coord = None
    lon_coord = None
    
    for name in lat_names:
        if name in ds.coords:
            lat_coord = name
            break
    
    for name in lon_names:
        if name in ds.coords:
            lon_coord = name
            break
    
    if lat_coord and lon_coord:
        print(f"Found geographic coordinates: {lat_coord}, {lon_coord}")
        
        # Create geographic plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Plot 1: Scatter plot of positions
        ax1.scatter(ds[lon_coord], ds[lat_coord], alpha=0.6, s=20)
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Latitude')
        ax1.set_title('Geographic Distribution')
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Density plot if enough points
        if len(ds[lat_coord]) > 50:
            ax2.hist2d(ds[lon_coord], ds[lat_coord], bins=20, alpha=0.8)
            ax2.set_xlabel('Longitude')
            ax2.set_ylabel('Latitude')
            ax2.set_title('Position Density')
        else:
            ax2.scatter(ds[lon_coord], ds[lat_coord], alpha=0.6, s=50)
            ax2.set_xlabel('Longitude')
            ax2.set_ylabel('Latitude')
            ax2.set_title('All Positions')
        
        plt.tight_layout()
        plt.show()
        
        # Print coordinate ranges
        print(f"Latitude range: [{ds[lat_coord].min().values:.4f}, {ds[lat_coord].max().values:.4f}]")
        print(f"Longitude range: [{ds[lon_coord].min().values:.4f}, {ds[lon_coord].max().values:.4f}]")
    
    else:
        print("No geographic coordinates found")


In [7]:
def analyze_variables(ds):
    """Statistical analysis of dataset variables"""
    
    print("="*60)
    print("STATISTICAL SUMMARY")
    print("="*60)
    
    for var_name, var in ds.data_vars.items():
        if np.issubdtype(var.dtype, np.number):
            print(f"\n{var_name}:")
            print(f"  Shape: {var.shape}")
            print(f"  Min: {var.min().values}")
            print(f"  Max: {var.max().values}")
            print(f"  Mean: {var.mean().values}")
            print(f"  Std: {var.std().values}")
            
            # Count NaN values
            if hasattr(var.values, 'isnan'):
                nan_count = np.isnan(var.values).sum()
                print(f"  NaN values: {nan_count} ({nan_count/var.size*100:.1f}%)")

def plot_time_series(ds, time_var=None):
    """Plot time series if time variable exists"""
    
    # Find time variable
    if time_var is None:
        time_candidates = ['JULD_datetime', 'time', 'TIME', 'JULD']
        for candidate in time_candidates:
            if candidate in ds.coords or candidate in ds.data_vars:
                time_var = candidate
                break
    
    if time_var is None:
        print("No time variable found for time series plotting")
        return
    
    print(f"Creating time series plots using: {time_var}")
    
    # Get numeric variables for plotting
    numeric_vars = [name for name, var in ds.data_vars.items() 
                   if np.issubdtype(var.dtype, np.number) and var.size > 1]
    
    if len(numeric_vars) == 0:
        print("No suitable variables for time series")
        return
    
    # Plot first few variables
    vars_to_plot = numeric_vars[:4]
    
    fig, axes = plt.subplots(len(vars_to_plot), 1, figsize=(12, 3*len(vars_to_plot)))
    if len(vars_to_plot) == 1:
        axes = [axes]
    
    for i, var_name in enumerate(vars_to_plot):
        try:
            if time_var in ds[var_name].dims:
                ds[var_name].plot(ax=axes[i])
            else:
                # Plot against time coordinate
                axes[i].plot(ds[time_var], ds[var_name])
                axes[i].set_xlabel(time_var)
                axes[i].set_ylabel(var_name)
            
            axes[i].set_title(f'{var_name} vs {time_var}')
            axes[i].grid(True, alpha=0.3)
            
        except Exception as e:
            axes[i].text(0.5, 0.5, f'Error: {str(e)[:50]}...', 
                        ha='center', va='center', transform=axes[i].transAxes)
    
    plt.tight_layout()
    plt.show()
